# RENDU EVALUATION KAISENDATA

| Prénoms       |     Nom         |     Email          |                                     |     Lieu    |   Date   |
| ------------- |: -------------: |--------------------|                                     | ------------|----------|
| Amadou lamarana      | DIALLO               | mailsdiallo@gmail.com|                       | Paris       | 16-06-2021|


<hr>
 

> Dans le cadre de cette évaluation des compétences  pour une demande de stage en NLP et Deep Learning, nous verrons en détails les procédures employées et les resultats obtenus. Il a été demande des récupérer des contenus, posts,sur facebook sur le décès de l'ancien Présdient, Jacques Chirac en 2019 .Le notebook est long dans le souci d'expliciter les méthodes pour plus de clarté. Je serai bien aise d'avoir un feedback et les axes d'améliorations possibles.

<hr>

### ETAPES :

#### A ) Importer les modules nécessaires

#### B ) Configuration  et lancement du navigateur webdriver

#### C ) La connexion sur le compte facebook pour récupérer les données ou posts

#### D) Parcours de toute la page avec un système de scrolling afin de récuperer tous les posts pertinents par rapport à notre thématique "le décès du président Jacques Chirac" et les résultats sont affichés sur la page

#### E) Récupération des articles des tous les posts sur la page de résultat 

#### F) Récupérer les liens des posts

#### G) Création d'un DataFrame df pour la persistance des liens

#### H) Sauvegarder les liens dans un fichier au format csv

#### I) Utilisation du package facebook-scraper

#### J) MongoDB pour la sauvegarde des données en local

#### A ) Importer les modules nécessaires

    * Selenuim :https://selenium-python.readthedocs.io/index.html

    * time et math uuid sont natifs à python lors de son l'installation

    * pandas: https://pandas.pydata.org/

    * cleantext : https://pypi.org/project/clean-text/


In [736]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import math
import uuid
##import pandas 
import pandas as pd
from cleantext import clean

#### B ) Configuration  et lancement du navigateur webdriver


##### Identifiant de connexion et mot de passe .. à personnaliser

In [528]:
identifiant ="amadou-lamarana.diallo.edu@groupe-gema.com"
mot_de_passe ="XXXXXXXX"
credentials= (identifiant, mot_de_passe)

In [737]:
""" Nous avons la configuration du webdriver en local avec 
les options élémentaires.
"""
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_argument('--incognito')
chrome_options.add_experimental_option("prefs",prefs)
"""
lien du webdriver :chrome
"""
exec_path = "./chromedriver"
"""
Exécution du webdriver
"""
browser = webdriver.Chrome(executable_path=exec_path, options=chrome_options)
"""
Accéder au site facebook
"""
browser.get("https://www.facebook.com")

"""
Acceptation des cookies pour continuer la simulation de connexion
"""
WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Tout accepter']"))).click()

#### C ) La connexion sur le compte facebook pour récupérer les données ou posts
La fonction login() aide à se connecter en utilisant des paramètres de connexion valides. 

In [702]:
def login():
    """
    Nous definissons d'une fonction login afin de se connecter sur facebook 
    avec les credentials ci dessous.
    Les taches nécessaires sont majoirtairement automatiques. 
    Le système reconnait les champs requis et fait lui même le remplissage. 
    """
    #target username
    """
    Detecter puis Renseigner l'identifiant (email et mot de passe)
    """
    email = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
    password = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))
    """
    Toujours vider les champs inputs grace au méthode clear() avant le remplissage
    """
    email.clear()
    #email = driver.find_element_by_id("email")
    identifiant =credentials[0]
    mot_de_passe =credentials[1]
    """
    Renseigner le username
    """
    email.send_keys(identifiant)
    #Password = driver.find_element_by_id("pass")
    password.clear()
    """
    Renseigner le mot de passe
    """
    password.send_keys(mot_de_passe)
    button = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    """
    connexion effectuée.. puis libérer 
    """
    pass 


#### La connexion proprement dite

In [738]:
"""
Puis action de se connecter sur Facebook 
"""
login()


#### D) Parcours de toute la page avec un système de scrolling afin de récuperer tous les posts pertinents par rapport à notre thématique "le décès du président Jacques Chirac" et les résultats sont affichés sur la page

In [739]:
"""
Les données sont filtrées d'avance sur Facebook et classé par Année, Publication publiques, etc. 
Nous allons définir le lien essentiel pour le résultat directement, 
"""
url_complet='https://www.facebook.com/search/posts?q=le%20d%C3%A9c%C3%A9s%20du%20pr%C3%A9sident%20%20jacques%20chirac&filters=eyJycF9hdXRob3I6MCI6IntcIm5hbWVcIjpcIm1lcmdlZF9wdWJsaWNfcG9zdHNcIixcImFyZ3NcIjpcIlwifSIsInJwX2NyZWF0aW9uX3RpbWU6MCI6IntcIm5hbWVcIjpcImNyZWF0aW9uX3RpbWVcIixcImFyZ3NcIjpcIntcXFwic3RhcnRfeWVhclxcXCI6XFxcIjIwMTlcXFwiLFxcXCJzdGFydF9tb250aFxcXCI6XFxcIjIwMTktMVxcXCIsXFxcImVuZF95ZWFyXFxcIjpcXFwiMjAxOVxcXCIsXFxcImVuZF9tb250aFxcXCI6XFxcIjIwMTktMTJcXFwiLFxcXCJzdGFydF9kYXlcXFwiOlxcXCIyMDE5LTEtMVxcXCIsXFxcImVuZF9kYXlcXFwiOlxcXCIyMDE5LTEyLTMxXFxcIn1cIn0ifQ%3D%3D'

"""
Nous tentons d'accéder au lien "url_complet"
"""
browser.get(url_complet)

"""methode scrolling pour récuperer plus de données dans la page"""

last_height = browser.execute_script("return document.body.scrollHeight") 
"""
Parcours autant que nécessaire jusqu'a atteindre le fond de la page
"""
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    """
    Attendre 2 secondes '2 s' avant chaque scroll vers le bas
    """
    time.sleep(2)
    """
    Recuperer la taille courante du frame
    """
    new_height = browser.execute_script("return document.body.scrollHeight")
    """
    Si la taille courante est égale à la taille maximale alors sortir 
    de boucle grâce une instrcution break
    """
    if new_height == last_height:
        break
    last_height = new_height

#### E) Récupération des articles des tous les posts sur la page de résultat 

In [740]:
"""
recuepérer les webElement(qui contient les liens informations du post) des posts de la page affichée
"""
#recherche =browser.find_elements_by_xpath('//*[@id="mount_0_0_wk"]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div[2]')
recherche = browser.find_elements(By.TAG_NAME, 'div[role="article"]')

In [488]:
recherche

[<selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="a74b7cf6-7805-40e8-a07e-aedbb44dcefe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="2765209e-fac9-4fd2-864f-682a2c048eaa")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="0e2ddaf7-b8a2-4f17-b0ae-5f1547f49108")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="e903d535-62df-4594-b880-dc4a05b5d5d3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="e7f3a9db-9914-4c9c-a694-2b6198115ac5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="c4407d49-c6cc-40a4-93e7-ab832a4b500c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b2782ff44ecf2320f089cd2967605f9f", element="3aff4133-e639-4922-852c-4f

#### F) Récupérer les liens des posts

In [489]:
"""
Nous allons récupérer les liens des posts pertinents chargés dans une liste
"""
data = [] ## liste des liens
"""
Boucle sur les posts pertinents déja existants
"""
for r in recherche:
    """
    les balises "a" contenant les liens des posts. ce que nous cherchons: 'anchors'
    """
    
    anchors = r.find_elements_by_tag_name('a[role="link"]')[-1]
    "recuperer la valeur de href: lien du post"
    link = anchors.get_attribute('href')
    
    """
    Ne récuperer que les liens posts
    """
    if "posts" in link:
        data.append(link)

#### G) Création d'un DataFrame df pour la persistance des liens

In [490]:
"""
Stocker les données dans un dataframe afin d'une prochaine utilisation
"""
df = pd.DataFrame(data, columns=['Link'])

In [491]:
"""
Suppresion des duplicats
"""
df.drop_duplicates(inplace=True)

In [492]:
df

,Link
0,https://www.facebook.com/blm27/posts/263308183...
1,https://www.facebook.com/FRANCE24/posts/101571...
2,https://www.facebook.com/yhijnk/posts/33201637...
3,https://www.facebook.com/RTIinfo225/posts/7744...
4,https://www.facebook.com/RadioOmegafmOfficiel/...
...,...
56,https://www.facebook.com/Focusinfos.net/posts/...
57,https://www.facebook.com/ActualitesCongoRDC/po...
58,https://www.facebook.com/lacharentemaritime/po...
59,https://www.facebook.com/Tunisie.Numerique/pos...


#### H) Sauvegarder les liens dans un fichier au format csv

In [493]:
df_read =df.to_csv("Liens_Post_Facebook.csv")

In [712]:
df_read

,Link
0,https://www.facebook.com/blm27/posts/263308183...
1,https://www.facebook.com/FRANCE24/posts/101571...
2,https://www.facebook.com/yhijnk/posts/33201637...
3,https://www.facebook.com/RTIinfo225/posts/7744...
4,https://www.facebook.com/RadioOmegafmOfficiel/...
...,...
56,https://www.facebook.com/Focusinfos.net/posts/...
57,https://www.facebook.com/ActualitesCongoRDC/po...
58,https://www.facebook.com/lacharentemaritime/po...
59,https://www.facebook.com/Tunisie.Numerique/pos...


#### I) Utilisation du package facebook-scraper

Lien web: https://pypi.org/project/facebook-scraper/


Installation :  pip install facebook-scraper

In [526]:
from facebook_scraper import get_posts

##### 1) Renseigner le login et le mot du compte qui sera utilisé pour se connecter sur Facebook

##### 2) Module de gestion de fichier et du telechargement des images en local

In [565]:
import os
import wget

#### J) Recupération des données sur Facebook  à partir des données
###### Les données à utiliser:  
    *** notre liste d'url disponible sur avec la variable data OU  
    *** récuperer les liens au niveau du dataframe  OU  
    *** par le fichier csv

In [717]:
data
#df= pd.read_csv('Liens_Post_Facebook.csv')
#data = list(df['Link'])


61

##### Utilisation du package facebook-scraper

In [720]:
"""
dictionnaire posts pour enregistrer les données..
"""
posts ={'texts':[],'image_link':[],'image_local_path':[], 'comments':[]}
"""
boucle sur les liens de données recupérées
"""
for post in get_posts(post_urls=data, credentials=credentials,options={"comments": True}):
    """
        text_NA: donnée à enregistrer en cas d'absence de texte
    """
    text_NA ='XXX'
    """
        si le texte du post est défini
    """
    if post['text'] is not None:
        """
            nettoyage du texte et ajout sur la liste du dictionnaire posts['texts']
        """
        posts['texts'].append(clean(post['text'].strip(),normalize_whitespace=True, no_line_breaks=True, lang='fr'))
    else:
        """
            si pas de text
        """
        posts['texts'].append(text_NA)
    """
        Si l'image est définie alors traiement et gestion de l'image
    """
    if post['image'] is not None:
        
        posts['image_link'].append(post['image'])
        """
            Télécharger l'image si elle est disponible
            # 1 : dossier de travail
            # 2 : mention du dossier de sauvegarde des images en local (dossier images_facebook)
            # 3 : si le dossier n'existe pas, alors le créer
            # 4 : filename du fichier avec l'id du post_id du post sur facebook combiné à un uuid() pour eviter les duplications
        """
        path = os.getcwd() # 1
        path = os.path.join(path, 'images_facebook') # 2
        if  os.path.exists(path)== False: # 3
            os.mkdir(path) # 3
        rename_file = os.path.join(path, str(post['post_id'])+str(uuid.uuid1())+'.png') # 4
        """
            Si le fichier n'existe pas en répertoire 
        """
        if os.path.exists(rename_file) == False:
            try:
                """
                    tentative de telechargement de l'image et sauvgarde sur un répertoire crée à l'occasion
                """
                wget.download(post['image'], rename_file)
                
                posts['image_local_path'].append(rename_file)
                
            except:
                """Cas où le téléchargement a échoué, j'ai jugé nécessaire de mettre en placeholder, 
                une image 'no_image' pour sympbiliser l'absence de l'élément
                """
                posts['image_local_path'].append(path+'/images_facebook/no_image.png')
                
    else:
        """
             Si il n'y a pas d'image.alors renseigner XXX sur le lien de notre dictioonaire. et pour lien d'image en local, l'image par défaut
        """
        posts['image_link'].append('XXX')
        posts['image_local_path'].append(path+'/images_facebook/no_image.png')
        
    """
        Si les commentaires existent alors: 
            1) Opération de nettoyage du texte
            2) Mettre tous les commentaires du post dans une même liste pour faciliter le traitement futur
    """
    if post['comments_full'] is not None:
        comment_sanitized =[]
        """
            Parcourir les commentaires du post
        """
        for comment in post['comments_full']:
            """
            Nettoyage du texte des commentaires et sauvegader dans une liste "comment_sanitized"
            """
            comment_sanitized.append(clean(comment['comment_text'],normalize_whitespace=True, no_line_breaks=True, lang='fr'))
        
        """
            Une fois la boucle terminée, la liste des commentaires nettoyés est ajouté dans notre dictionnaire 'posts' au niveau de la liste "comments"
        """
        posts['comments'].append(comment_sanitized)
        """
            Vider la liste des commentaires nettoyés du post et passage pour le prochain post
        """
        comment_sanitized =[]
    else:
        """
            Si il n'y a pas de commentaire pour le post alors .. la valeur 'XXX' lui est attribuée
        """
        posts['comments'].append('XXX')

In [721]:
posts

{'texts': ['c\'est dans l\'avion de retour de rodez que j\'ai appris le deces du president jacques chirac. il aimait ce departement de l\'aveyron, comme il aimait tous les departements ruraux, dont il avait la force et la simplicite. j\'avais 29 ans quand j\'ai commence a travailler a ses cotes et, au moment si triste de sa mort, les souvenirs se bousculent. pourquoi, comme tant de francais, ai-je aime jacques chirac ? je l\'ai aime pour ses mots lors de mon premier salon de l\'agriculture en 2009 quand, presse de questions par les journalistes sur sa passion pour le monde agricole, il avait repondu, deja fatigue par la maladie : " j\'aime l\'agriculture, parce que l\'agriculture, c\'est la vie. " je l\'ai aime pour sa voix quand, jeune directeur de cabinet de dominique de villepin a matignon, il m\'appelait chaque matin a huit heures precises, avec cette intonation si douce et si rauque : " le maire ? c\'est chirac. " j\'ai aime jacques chirac pour sa constance a dire non aux extremes

In [722]:
posts['comments'][0]


['un grand hommage a ce president qui allait beaucoup a la rencontre des francais',
 'un bel hommage sincere et touchant en toute simplicite.',
 'ce qui me navre dans cette photo c de voir l autre tare cote de jacques chirac',
 "c'est un tres bel hommage bruno ! que le temps passe vite , que la vie passe vite !... je l'aimais egalement beaucoup , pour sa chaleur , son courage et ses convictions, son amour de la france et des francais ( j'ai l'intime sentiment qu'il aimait vraiment les etres humains ), ses mots et sa gourmandise! je crois que sa plus grande crainte etait que les francais soient divises...il a ete le premier a utiliser le terme de fracture sociale ... un monsieur, un homme digne nous quitte. un morceau de ma jeunesse s'en va avec lui ... je suis bien triste de sa disparition. mes pensees vont a sa famille et ses amis proches",
 "tres bel hommage. vous nous faites ressortir les valeurs de ce grand monsieur. vous pouvez comparer le respect d'une collection au quotidien. je

In [723]:
posts['comments']

[['un grand hommage a ce president qui allait beaucoup a la rencontre des francais',
  'un bel hommage sincere et touchant en toute simplicite.',
  'ce qui me navre dans cette photo c de voir l autre tare cote de jacques chirac',
  "c'est un tres bel hommage bruno ! que le temps passe vite , que la vie passe vite !... je l'aimais egalement beaucoup , pour sa chaleur , son courage et ses convictions, son amour de la france et des francais ( j'ai l'intime sentiment qu'il aimait vraiment les etres humains ), ses mots et sa gourmandise! je crois que sa plus grande crainte etait que les francais soient divises...il a ete le premier a utiliser le terme de fracture sociale ... un monsieur, un homme digne nous quitte. un morceau de ma jeunesse s'en va avec lui ... je suis bien triste de sa disparition. mes pensees vont a sa famille et ses amis proches",
  "tres bel hommage. vous nous faites ressortir les valeurs de ce grand monsieur. vous pouvez comparer le respect d'une collection au quotidie

In [724]:
posts['texts'][4]

'deces jacques chirac : " l\'afrique perd un ami et un partenaire formidable " (roch kabore, president du faso) le president du faso roch kabore a reagi au deces jeudi de l\'ancien president francais jacques chirac (1995-2007) et premier ministre (1974-1976, 1986-1988). " je viens d\'apprendre avec tristesse le deces de l\'ancien president francais jacques chirac. avec sa disparition, l\'afrique perd un ami et un partenaire formidable. je presente mes sinceres condoleances a sa famille, au peuple francais et emmanuel macron ", a-t-il ecrit sur son compte twitter.'

##### Réorganisation des commentaires sous format liste de dictionnaires
<code>    post :{
                'text': le_texte,
                'image': l_image_local,
                'comments': [{comment1},{comment_2},......]
            } </code>

In [725]:
"""
Nous allons organiser les commentaires pour chaque post, sous le format ci-dessus. 
pour facilter la representaion et le stockage en mongoDB
"""
comments = {}
all_comments =[]
for k in range(len(posts['comments'])):
    for com in posts['comments'][k]:
        comment = {'comment_text': com }
        all_comments.append(comment)
    comments[k] = all_comments
    all_comments = []


In [726]:
comments[5]

[{'comment_text': 'bande de clowns ces chirac ! pas de mlp, pas de la fille adoptive de jacques chirac mais des milliers d,anonymes qui n\'en ont certainement rien a faire de j. chirac, qui refusent 5 mn de queue aux caisses des supermarches, 10 minutes chez le docteur, un quart d\'heure aux urgences pour leurs petits bobos mais qui attendent 6 heures dans le vent et sous la pluie pour se prendre en selfie avec un cercueil et signer un livre que personne ne lira ! contraire a jacques chirac qui etait parait il un " grand rassembleur " ce qui n\'est pas le cas de sa famille 👎👎'},
 {'comment_text': "moi on ma toujours dit que l'eglise appartient a tous le monde marine le pen a le droit dis aller c'est un hommage populaire citoyens citoyennes quelle bandes de clowns on voit bien ou le pognon des francais sont utilises vive marine le pen pour 2022"},
 {'comment_text': 'les gens on la memoire courte qui a fait voter maastricht c est chirac quand meme pourtant les francais n en voulait pas e

#### J) MongoDB pour la sauvegarde des données en local

In [727]:
data

['https://www.facebook.com/blm27/posts/2633081830089213',
 'https://www.facebook.com/FRANCE24/posts/10157122109541936',
 'https://www.facebook.com/yhijnk/posts/3320163724690564',
 'https://www.facebook.com/RTIinfo225/posts/774441316326061',
 'https://www.facebook.com/RadioOmegafmOfficiel/posts/3128376093901396',
 'https://www.facebook.com/MarineLePen/posts/3021864544496516',
 'https://www.facebook.com/PJugnauth/posts/2572593699450087',
 'https://www.facebook.com/GuillaumeKsoro/posts/2432114566903241',
 'https://www.facebook.com/Le360.ma/posts/1464780243675767',
 'https://www.facebook.com/RadioOmegafmOfficiel/posts/3127433227329016',
 'https://www.facebook.com/LADFInfos/posts/132825348079140',
 'https://www.facebook.com/KarimKeita.ML/posts/2717611924936718',
 'https://www.facebook.com/leprogres.saintetienne/posts/2313056118791924',
 'https://www.facebook.com/equinoxe.television/posts/2528870197384724',
 'https://www.facebook.com/partisocialiste/posts/10158871690378957',
 'https://www.fa

In [675]:
"""
Nous allons utiliser le package pymongo pour interagir avec MongoDB installé en local.
Les paramètres sont ceux définis par défaut .. sans user sans mot de passe
pprint sert à un afficahge pour une facilité de lecture et de visualisation.
"""
from pymongo import MongoClient
from pprint import pprint
"""
Etape1: Se connecter à la base MongoDB
"""
client = MongoClient(port=27017)
"""
Création de la base de données kaisendata
"""
db=client.kaisendata

"""
Etape 2: Schema de données
"""
n = len(posts['texts']) ## nombre de posts 
"""
parcours de 0 à n-1
le but de cet ensemble d'istruction est de définir un dictionnaire simple pour chaque post pour des rasions
 **** facilité l'affichage et les insertions dans la base de données mongodb
"""
for i in range(n):
    post ={
        'text' : posts["texts"][i],
        'image':posts['image_local_path'][i],
        'comments':comments[i]
    }
    #print(comments[i])
    """
    Etape 3: Insertion de chaque post directement dans la base de données db.kaisendata
    """
    result=db.kaisendata.insert_one(post)


#### Liste de bases de données en local sur mongoDB

In [728]:
client.database_names()

<ipython-input-728-896185ff0974>:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  client.database_names()


['admin', 'config', 'inventory', 'kaisendata', 'local', 'test']

#### Petite manipulation pour la visualisation des données

In [681]:
"""
Affichage des documents
"""
for d in db.kaisendata.find():
    pprint(d)

{'_id': ObjectId('60c958664915db6f1a8a2a2c'),
 'comments': [{'comment_text': 'un grand hommage a ce president qui allait '
                               'beaucoup a la rencontre des francais'},
              {'comment_text': 'un bel hommage sincere et touchant en toute '
                               'simplicite.'},
              {'comment_text': 'ce qui me navre dans cette photo c de voir l '
                               'autre tare cote de jacques chirac'},
              {'comment_text': "c'est un tres bel hommage bruno ! que le temps "
                               'passe vite , que la vie passe vite !... je '
                               "l'aimais egalement beaucoup , pour sa chaleur "
                               ', son courage et ses convictions, son amour de '
                               "la france et des francais ( j'ai l'intime "
                               "sentiment qu'il aimait vraiment les etres "
                               'humains ), ses mots et sa gou

              {'comment_text': "pour rendre hommage on oublie qu'on fait parti "
                               'de tel ou tel partie on oublie la politique '
                               "c'est petit quand meme"},
              {'comment_text': 'la trahison de nicolas sarkozy pour balladur '
                               "et tant d'autres ....et pourtant eux seront "
                               'present a la ceremonie .... ridicule tout '
                               'cette foutaise'},
              {'comment_text': "on s'en fou ,elle vaut mieux que ca ... et la "
                               'plus part de ceux qui etaient sont des '
                               'hypocrites !!!'},
              {'comment_text': "une femme exclue je crois qu'on n'est pas pret "
                               "pour l'egalite mais quand meme 12 millions de "
                               'francais qui ont vote pour le rn'},
              {'comment_text': 'madame marine le pen vous avez mon s

                               'promis un retour et depuis sans suite. '
                               'certains gabonais ont peine pour faire des '
                               'economies et vous, vous la coulez douce sur la '
                               'soeur des fronts des gabonais. vous vivez bien '
                               'et nous dans la misere. le gabonais veut son '
                               'argent donnez nous. les economies de nos '
                               "comptes epargne postaux detourne par l'etat "
                               "gabonais et ces dirigeants trop c'est trop. "
                               'nous voulons remboursement des soldes comptes '
                               'postaux au gabon'},
              {'comment_text': 'mr le president avc tout le respect que je '
                               "vous dois vaut mieux considere quelqu'un qui "
                               'te dit la verite au lieu de ceux la qui font '
           

                               "d'assuree."},
              {'comment_text': 'sinceres condoleances a sa famille et ces '
                               'amis'},
              {'comment_text': 'le tapi que c a feme vole et vendu la onte'},
              {'comment_text': 'poteva vivere se non avrebbe fatto le sue '
                               'cazzate adesso che riposa in pace cosa dira '
                               'davanti alla faccia di dio'},
              {'comment_text': 'tina guegan'},
              {'comment_text': 'magnifique photo'},
              {'comment_text': "un grand president s'en va..."},
              {'comment_text': 'respect'},
              {'comment_text': '"c\'est beau mais c\'est loin" la phrase '
                               'culte'},
              {'comment_text': 'on est toujours parfait quand on est mort'},
              {'comment_text': 'toute une epoque....'},
              {'comment_text': "un president d'une bonne epoque ,,paix a son "
         

              {'comment_text': 'la pure realite de notre societe '
                               'actuelle.....'},
              {'comment_text': 'pfff... a regarder pour mieux comprendre la '
                               'manipulation mediatique'},
              {'comment_text': "thinkerview bien dit! 👍🏻 c'est quoi cette "
                               "histoire de banque d'ailleurs ?"},
              {'comment_text': 'encore un coup de greenpeace selon '
                               'castagnette'},
              {'comment_text': "mais avec tout cet hommage est ce que l'on "
                               'sait si manger des pommes est bon pour la '
                               'sante ?'},
              {'comment_text': 'en verite, je vois pas mal de billets pointer '
                               "ca, mais ce n'est pas incoherent. il y a une "
                               'information de fond qui va avoir ses '
                               'developpement pendant pas mal 

                               'oublie les emplois fictifs de la mairie de '
                               'paris ? son soutien indefectible aux pouvoirs '
                               'sanguinaires en afrique ?'},
              {'comment_text': 'a sa famille, a ses proches a la radio foot '
                               "internationale et a l'equipe de france "
                               "j'associe mes prieres pour le repos de son "
                               'ame'},
              {'comment_text': "quand paris gagne le real madrid c'est la "
                               'victoire des joueurs, quand ils perdent hier '
                               'la defaite incombe a l entraineur. ..'},
              {'comment_text': "hommages a jaques chirac, c'etait un homme "
                               'exceptionnel!!'},
              {'comment_text': 'salut ani suis tres decu de sylvinho tres tres '
                               "decu j'espere qu'ils va reconnaitre c'est "
   

#### Nombre de documents dans la base

In [729]:
mesdonnes = db.kaisendata

In [732]:
mesdonnes.count_documents({})

61

<div class='text text-primary'> Nos documents sont bien enregistrés dans la base mongoDB </div>

### CONCLUSION

Nous arrivons à terme des objectifs fixés par cette évaluation. Nous avons implémenté des scripts à l'aide Selenuim, et le package facebook-scraper pour la recherche de l'information à savoir les posts, les images et les commentaires associés sur Facebook sur la thématique  'le décès du président Jacques Chirac' en septembre 2019.
Nous avons ensuite pû implémenter le script pour l'enregistrement dans une base de données MongoDB. 
Nous avons pas utilisé une application Facebook pour se servir des GraphAPI de Facebook, vu que la thématique demandé n'est pas lié à une page quelconque. Ce qui nous a amené à implementer un script de scrapping à l'aide Sélénuim.